In [19]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### BEESKOW ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/Großetranke_wehr_up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Grosse Trünke WehrU.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Grosse Trünke WehrUP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="white", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)



# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grosse Trünke WehrUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=500, y=250, x_units='screen', y_units='screen',
              text='NS=0.84',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=500, y=230, x_units='screen', y_units='screen',
              text='R²=0.85',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
              text='KGE=0.91',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 40  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.1)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)


# Große Tranke Wehr UP

In [ ]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/Großetranke_wehr_up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Grosse Trünke WehrU.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Grosse Trünke WehrUP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
#simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+40)
#p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)
p.y_range = Range1d(0, df_beeskow['diversion'].max()+40)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+50, end=0)}
precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
#hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_diversion,hover_precip)
p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grosse Trünke WehrUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=210, x_units='screen', y_units='screen',
              text='NS=0.1',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=190, x_units='screen', y_units='screen',
              text='R²=0.5',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=170, x_units='screen', y_units='screen',
              text='KGE=0.19',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"

# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



# Calculate the trend line equation for the diverted data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["diversion"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="blue", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# Hohenbinde

In [43]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/Hohenbinde_final.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Grosse Trünke WehrU.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Hohenbinde", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Hohenbinde", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=210, x_units='screen', y_units='screen',
              text='NS=0.17',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=190, x_units='screen', y_units='screen',
              text='R²=0.49',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=170, x_units='screen', y_units='screen',
              text='KGE=0.29',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"

# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



# Calculate the trend line equation for the diverted data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="blue", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 24_Frustenwalde OP

In [137]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/24_Frustenwaldeop.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Fuerstenwalde-OP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24']), end=0)}
precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Fuerstenwalde-OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.80',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.84',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.88',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.08
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 99_Beeskow

In [135]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/99_Beeskow.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="BeeskowUP, Spreeschleuse", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"99": Range1d(start=max(df_precip['99']), end=0)}
precip_glyph=p.vbar(x="Date", top="99", width=0.5, color='skyblue', source=source_precip, y_range_name="99", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="99",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="BeeskowUP, Spreeschleuse", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.88',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.89',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.94',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.08
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# Beeskow

In [77]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/Beeskow.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Beeskow.html")

# Create a figure object with title, axis labels and tools
p = figure(title="BeeskowUP Spreeschleuse", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+50, end=0)}
precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="BeeskowUP Spreeschleuse", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=210, x_units='screen', y_units='screen',
              text='NS=0.86',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=190, x_units='screen', y_units='screen',
              text='R²=0.87',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=170, x_units='screen', y_units='screen',
              text='KGE=0.92',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="9pt"
######################## Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

###################### Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

################Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 108_Ressen Stau UP

In [134]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/108_Ressen Stau Up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Ressen Stau UP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"108": Range1d(start=max(df_precip['108'])+3, end=0)}
precip_glyph=p.vbar(x="Date", top="108", width=0.5, color='skyblue', source=source_precip, y_range_name="108", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="108",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Ressen Stau UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.55',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.42',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.25',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)



# 23_Berkenbrück2

In [133]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/23_Berkenbrucke2.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Berkenbrück2", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"23": Range1d(start=max(df_precip['23'])+3, end=0)}
precip_glyph=p.vbar(x="Date", top="23", width=0.5, color='skyblue', source=source_precip, y_range_name="23", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="23",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Berkenbrück2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.21',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.28',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.12',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 14_Kienbaum SportschuleUP

In [130]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/14_Kienbaum_Sportschule.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Kienbaum SportschuleUP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.8)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.8)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"14": Range1d(start=max(df_precip['14'])+3, end=0)}
precip_glyph=p.vbar(x="Date", top="14", width=0.5, color='skyblue', source=source_precip, y_range_name="14", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="14",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kienbaum SportschuleUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS=0.19',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.26',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE=0.09',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 9_Fredersdorf

In [129]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/9_Fredersdorf.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Fredersdorf", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.8)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.8)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"9": Range1d(start=max(df_precip['9'])+3, end=0)}
precip_glyph=p.vbar(x="Date", top="9", width=0.5, color='skyblue', source=source_precip, y_range_name="9", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="9",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Fredersdorf", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.40',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.04',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE=0.17',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 11_Dahlwitz Hopper Garten

In [128]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/11_Dahlwitz_Hoppergarten.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Dahlwitz-Hoppegarten", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()-3)
p.y_range = Range1d(0, df_beeskow['simulated'].max()-3)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"11": Range1d(start=max(df_precip['11']), end=0)}
precip_glyph=p.vbar(x="Date", top="11", width=0.5, color='skyblue', source=source_precip, y_range_name="11", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="11",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Dahlwitz-Hoppegarten", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -2.24',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.14',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"



#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 12_Heidekrug

In [124]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/12_Heidekrug.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Heidekrug", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"12": Range1d(start=max(df_precip['12']), end=0)}
precip_glyph=p.vbar(x="Date", top="12", width=0.5, color='skyblue', source=source_precip, y_range_name="12", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="12",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Heidekrug", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.37',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.06',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 19_Grunheide2 (not approved)

In [123]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/19_Grunheide2.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Grünheide2", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max()+4)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"19": Range1d(start=max(df_precip['19']+3), end=0)}
precip_glyph=p.vbar(x="Date", top="19", width=0.5, color='skyblue', source=source_precip, y_range_name="19", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="19",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grünheide2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -2.30',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.42',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.26',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 27_Berckenbrucke1

In [122]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/27_Berckenbrücke1.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Berkenbrück1", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.4)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.4)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"27": Range1d(start=max(df_precip['27']), end=0)}
precip_glyph=p.vbar(x="Date", top="27", width=0.5, color='skyblue', source=source_precip, y_range_name="27", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="27",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Berkenbrück1", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=300, y=200, x_units='screen', y_units='screen',
              text='NS= 0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=300, y=180, x_units='screen', y_units='screen',
              text='R²=0.24',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=300, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.02',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 56_Goyatz2

In [121]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/56_Goyatz2.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Goyatz2", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.2)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.2)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"56": Range1d(start=max(df_precip['56']), end=0)}
precip_glyph=p.vbar(x="Date", top="56", width=0.5, color='skyblue', source=source_precip, y_range_name="56", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="56",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Goyatz2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.14',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²=0.16',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.22',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 59_Pieskow

In [120]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/59_Pieskow.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Pieskow", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.2)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.2)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"59": Range1d(start=max(df_precip['59']), end=0)}
precip_glyph=p.vbar(x="Date", top="59", width=0.5, color='skyblue', source=source_precip, y_range_name="59", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="59",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Pieskow", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.32',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.23',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.21',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 60_Schulenwise

In [119]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/60_Schulenwise.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Schuhlen-Wiese", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.2)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.2)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"60": Range1d(start=max(df_precip['60']), end=0)}
precip_glyph=p.vbar(x="Date", top="60", width=0.5, color='skyblue', source=source_precip, y_range_name="60", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="60",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Schuhlen-Wiese", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.04',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.26',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 61_Mollen1


In [118]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/61_Mollen1.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Möllen 1", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()+0.2)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+0.2)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"61": Range1d(start=max(df_precip['61']), end=0)}
precip_glyph=p.vbar(x="Date", top="61", width=0.5, color='skyblue', source=source_precip, y_range_name="61", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="61",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Möllen 1", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.05',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.01',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.17',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.2
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 66_Doberburg

In [117]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/66_Dobberburg.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Doberburg", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"66": Range1d(start=max(df_precip['66']), end=0)}
precip_glyph=p.vbar(x="Date", top="66", width=0.5, color='skyblue', source=source_precip, y_range_name="66", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="66",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Doberburg", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.22',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.38',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 76_Eggersdorf

In [116]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/76_Eggersdorf.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Eggersdorf2", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"76": Range1d(start=max(df_precip['76']), end=0)}
precip_glyph=p.vbar(x="Date", top="76", width=0.5, color='skyblue', source=source_precip, y_range_name="76", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="76",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Eggersdorf2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.51',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.09',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.11',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 78_Altandsberg2

In [115]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/78_Altandsberg2.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Altlandsberg2, Walkmühle", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max()-1.5)
p.y_range = Range1d(0, df_beeskow['simulated'].max()-1.5)
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"78": Range1d(start=max(df_precip['78']), end=0)}
precip_glyph=p.vbar(x="Date", top="78", width=0.5, color='skyblue', source=source_precip, y_range_name="78", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="78",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Altlandsberg2, Walkmühle", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -1.61',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.10',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.07',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"

#################Calculate the trend line equation for the observed data
#observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#observed_y = np.array(df_beeskow["observed"])
#observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
#observed_trendline = (observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1])+1

#################Calculate the trend line equation for the simulated data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]



#################Calculate the trend line equation for the diverted data
#simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
#simulated_y = np.array(df_beeskow["simulated"])
#simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
#simulated_trendline = (simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1])+1

# Add the trend lines to the plot
#p.line(x=observed_x, y=observed_trendline, line_width=2.3, line_dash="dashed",color="blue", legend_label="Observed Trendline")
#p.line(x=simulated_x, y=simulated_trendline, line_width=2.3, color="yellow", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 79_Bruckmuhle wehr up

In [114]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/79_Bruckmuhle_wehr_up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Bruchmühle, WehrUP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"79": Range1d(start=max(df_precip['79']), end=0)}
precip_glyph=p.vbar(x="Date", top="79", width=0.5, color='skyblue', source=source_precip, y_range_name="79", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="79",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Bruchmühle, WehrUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= 0.07',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.21',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.35',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 81_schoneiche

In [111]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/81_Schoniche.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Schöneiche", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"81": Range1d(start=max(df_precip['81']+2), end=0)}
precip_glyph=p.vbar(x="Date", top="81", width=0.5, color='skyblue', source=source_precip, y_range_name="81", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="81",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Schöneiche", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -3.42',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.02',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.41',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 83_Woltersdorf

In [109]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/83_Woltersdorf.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Woltersdorf Schleuse OP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"83": Range1d(start=max(df_precip['83']), end=0)}
precip_glyph=p.vbar(x="Date", top="83", width=0.5, color='skyblue', source=source_precip, y_range_name="83", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="83",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Woltersdorf Schleuse OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -4.16',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.00',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.42',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 85_Kienbaum Straßenbrucker

In [108]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/85_Kienbaum Strasenbrucke.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Kienbaum,Straßenbrücke", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"85": Range1d(start=max(df_precip['85']), end=0)}
precip_glyph=p.vbar(x="Date", top="85", width=0.5, color='skyblue', source=source_precip, y_range_name="85", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="85",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kienbaum,Straßenbrücke", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -2.71',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.16',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.24',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 96_Kersdorf Schluse

In [107]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/96_Kersdorf schlueuse op.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Kersdorf Schleuse OP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"96": Range1d(start=max(df_precip['96']), end=0)}
precip_glyph=p.vbar(x="Date", top="96", width=0.5, color='skyblue', source=source_precip, y_range_name="96", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="96",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kersdorf Schleuse OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.75',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.01',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.05',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 100_Oegeln

In [145]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/100_Oegeln.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Oegeln", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"100": Range1d(start=max(df_precip['100']), end=0)}
precip_glyph=p.vbar(x="Date", top="100", width=0.5, color='skyblue', source=source_precip, y_range_name="100", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="100",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Oegeln", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -1.75',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.23',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.01',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 101_Greidland Panzerbrucker

In [139]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/101_Freiland Panzerbrucke.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Friedland Panzerbrücke", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"101": Range1d(start=max(df_precip['101']), end=0)}
precip_glyph=p.vbar(x="Date", top="101", width=0.5, color='skyblue', source=source_precip, y_range_name="101", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="101",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Friedland Panzerbrücke", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -1.63',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.07',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.02',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 102_Pretschen

In [141]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/102_Pretschen.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Pretschen", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"102": Range1d(start=max(df_precip['102']), end=0)}
precip_glyph=p.vbar(x="Date", top="102", width=0.5, color='skyblue', source=source_precip, y_range_name="102", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="102",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Pretschen", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -1.61',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.24',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= -0.03',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 109_Doberburg_Muhle_UP

In [142]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/109_Dobberburg Muhle Up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Doberburg Mühle UP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"109": Range1d(start=max(df_precip['109']), end=0)}
precip_glyph=p.vbar(x="Date", top="109", width=0.5, color='skyblue', source=source_precip, y_range_name="109", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="109",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Doberburg Mühle UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='NS= -0.76',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='R²= 0.13',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=160, x_units='screen', y_units='screen',
              text='KGE= 0.33',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 112_Lieberose_Wehr_OP

In [144]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/112_Lebeorse Wehr op.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Lieberose Wehr OP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"112": Range1d(start=max(df_precip['112']), end=0)}
precip_glyph=p.vbar(x="Date", top="112", width=0.5, color='skyblue', source=source_precip, y_range_name="112", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="112",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Lieberose Wehr OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=220, x_units='screen', y_units='screen',
              text='NS= -4.41',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='R²= 0.19',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='KGE= 0.15',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "center_left"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 91_Großetranke

In [149]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/91_Grosetranke Wehr up.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Grosse-Tränke-Wehr-UP", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"91": Range1d(start=max(df_precip['91']), end=0)}
precip_glyph=p.vbar(x="Date", top="91", width=0.5, color='skyblue', source=source_precip, y_range_name="91", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="91",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grosse-Tränke-Wehr-UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=220, x_units='screen', y_units='screen',
              text='NS= 0.33',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='R²= 0.65',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='KGE= 0.43',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)


# 92_Hohenbinde

In [151]:
#MODIFICATION INTO ONE CSV FILE FOR DISCHARGE
###################################### ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Discharge Bokeh Plotting/92_Hohenbinde.csv')
df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated Stream flow Kienbaum SportschuleUP.html")

# Create a figure object with title, axis labels and tools
p = figure(title="Hohenbinde", x_axis_label="Year", y_axis_label="Discharge (m³/s)",  x_axis_type="datetime",width=900,height=380, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 5
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="gray", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="red", name="simulated", legend_label="Simulated")
#diversion_glyph = p.line(x="Date", y="diversion", source=source_beeskow, line_width=2, color="red", name="diversion", legend_label="Diversion")
p.y_range = Range1d(0, df_beeskow['observed'].max())
p.y_range = Range1d(0, df_beeskow['simulated'].max())
#p.y_range = Range1d(-20, df_beeskow['diversion'].max()+50)


# create precipitation bar chart on right y-axis
p.extra_y_ranges = {"92": Range1d(start=max(df_precip['92']), end=0)}
precip_glyph=p.vbar(x="Date", top="92", width=0.5, color='skyblue', source=source_precip, y_range_name="92", legend_label='Precipitation')
p.add_layout(LinearAxis(y_range_name="92",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_diversion = HoverTool(tooltips=[("Diversion", "@diversion{0.00}")], renderers=[diversion_glyph])
hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
p.add_tools(hover_observed, hover_simulated,hover_precip)
#p.add_tools(hover_observed, hover_diversion,hover_precip)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Hohenbinde", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title


# Create a label object for ns
label_ns = Label(x=350, y=220, x_units='screen', y_units='screen',
              text='NS= 0.41',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=350, y=200, x_units='screen', y_units='screen',
              text='R²= 0.65',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
label_kge = Label(x=350, y=180, x_units='screen', y_units='screen',
              text='KGE= 0.48',text_align='center', text_color ='black',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
#p.legend.location = "bottom_center"
p.legend.location = "top_right"
#p.legend.orientation = "horizontal"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.07
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_font_size="10pt"
p.legend.label_text_color="black"


# remove grid lines in the plot area
p.grid.grid_line_color = 'white'

# Show the figure in a browser
show(p)
#save(p)
